In [ ]:
library(data.table)
library(dplyr)
library(tidyr)

In [ ]:
dfs <- lapply(snakemake@input$des,
       function(x){
           df <- fread(x) %>%
               select(qpid, desc, genename, val_avg, jac_avg) %>%
               mutate(
                    qpid = as.character(qpid),
                    `PANNZER desc. ` = as.character(desc),
                    `PANNZER genename` = as.character(genename),
                    val_avg=as.numeric(val_avg),
                    jac_avg = as.numeric(jac_avg)
               ) %>%
               select(-desc, -genename)
           return(df)
           # print(names(x))
       }
    
)

In [ ]:
dfs2 <- bind_rows(dfs) %>%
    group_by(qpid) %>%
    arrange(-val_avg, -jac_avg, .by_group = T) %>%
    slice_head(n=1) %>%
    ungroup()
dfs2

In [ ]:
# Load the data.table package
library(data.table)

# Define the file path
file_path <- snakemake@input$pro

# Read the entire file as a single column per line
data <- fread(file_path, header = FALSE, sep = "\n", col.names = "line")

# Directly create the qpid and remaining columns by finding the first space and splitting the string
data[, qpid := substr(line, 1, regexpr(" ", line) - 1)]
data[, remaining := substr(line, regexpr(" ", line) + 1, nchar(line))]

# Remove the original 'line' column
data[, line := NULL]

data <- data %>%
    rename(`PROKKA desc` = remaining)

In [ ]:
dfso <- data %>%
    mutate(qpid = sub(">","",qpid)) %>%
    left_join(dfs2, by = "qpid") %>%
    select(-val_avg, -jac_avg)
    
fwrite(x = dfso, file = snakemake@output[[1]])